<a href="https://colab.research.google.com/github/hwankang/Pytorch_tutorials/blob/main/hwankang_0818_adagrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline
import numpy as np
import pandas as pd

from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

# Extract sepal length, petal length from Setosa and Versicolor
data = iris.data[:100, [0, 2]]

# Standardization
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Split data to test and train data
train_x, test_x, train_y, test_y = train_test_split(data, iris.target[:100].reshape((-1, 1)), test_size=0.3)

print('train_x:', train_x.shape)
print('train_y:', train_y.shape)
print('test_x:', test_x.shape)
print('test_y:', test_y.shape)

# Plotting data
scatter(data[:50, 0], data[:50, 1], label='Setosa')
scatter(data[50:100, 0], data[50:100, 1], label='Versicolour')
title('Iris Data')
xlabel('sepal length')
ylabel('petal length')
grid()
legend()

In [ ]:
w = np.array([ 0.09370901, -0.24480254, -0.84210235]) # np.random.randn(2 + 1)
import matplotlib.pyplot  as plt
def predict(w, x):
    N = len(x)
    yhat = w[1:].dot(x.T) + w[0]
    yhat = yhat.reshape(-1)
    return yhat


def  adagrad_nn2(w, X, Y, eta=0.01, acmu_size=1):
    """
    @param eta <float>: learning rate
    @param acmu_size <int>: the size of the accumulation of the squred of the gradients with regard to theta
    """
    N = len(X)
    e = 1e-8
    G = np.zeros_like(w)

    i = 0
    for i in range(N):
        x = X[i]
        y = Y[i]
        x = x.reshape((-1, 2))
        yhat = predict(w, x)
        delta = y - yhat

        # Calculate the gradients
        gradient_w = 2/N*-delta.dot(x)
        gradient_b = 2/N*-delta

        G[1:] += gradient_w * gradient_w
        G[0] += gradient_b * gradient_b
        delta_w = eta/(e + np.sqrt(G[1:])) * gradient_w
        delta_b = eta/(e + np.sqrt(G[0])) * gradient_b
        w[1:] = w[1:] - delta_w
        w[0] = w[0] - delta_b

        if np.linalg.norm(w) < 1e-3:
            break
    return w
accu_ar=[]
for i in range(50):
    w = adagrad_nn2(w, train_x, train_y)
    yhats = predict(w, test_x)
    yhats = np.where(yhats >= 0.5, 1, 0)
    accuracy = accuracy_score(test_y, yhats)
    accu_ar.append(accuracy)
    print('Accuracy:', round(accuracy, 2), '\t\tweights:', w)
plt.figure(2)
plt.title("adagrad")
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt,plot(accu_ar, color='blue',label='adagrad', marker='o')
plt.legend(loc='lower righr')
grid()
plt.show()